## Twitter Sentiment Analyzer Program

## Libraries

In [1]:
import twitter
import nltk
import pandas as pd
import numpy as np
import pickle
import re as charRemoval
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
import time
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/mmvc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mmvc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Connecting to Twitter and Retrieving Data

In [2]:
#connecting to twitter server with credentials that we have after making a twitter developer account
twitter_api = twitter.Api(consumer_key='utm9PGkthAl86J16u9x01G4nJ',
                        consumer_secret='wN8OcAERDZ9xt7Hklh3K5Y4cJa2Ym2RQlvEv5opNfAo89DmahE',
                        access_token_key='1237428524845346816-7mMI2Fup908iLjENzOwZmAkn6fuOc4',
                        access_token_secret='V9Q5cBeVJBYO3ZKKoVRSnQq1EccW0Ws05advOFp7q2Au4',
                         tweet_mode='extended')

In [3]:
#function to retrieve tweets
def tweetsRetrieval(keyword):
    #get search function returns tweet based on key word
    retrievedTweets = twitter_api.GetSearch(keyword, count=100, lang='en', result_type='recent')
    print('Retrieved tweets for the keyword ({0}):  {1} \n'.format( key_word, len(retrievedTweets)))
    testset=[]
    #loopiing through the retrievedTweets and appending the text and the time it was created 
    for tw in retrievedTweets:
        testset.append({"text":tw.full_text, "label":None, "time":tw.created_at}) 
    return testset

## Importing Training Data Set

In [4]:
# Importing the training data set and loading it into a panda data frame
dataFramTraining = pd.read_csv("./Data/trainingdataset17k.csv", header=None, usecols=[0,5], names=['tweet polarity','text'], encoding="ISO-8859-1")
#grouping the data by labels
dataFramTrainingNegative = dataFramTraining.loc[dataFramTraining['tweet polarity']==0]
dataFramTrainingPositive = dataFramTraining.loc[dataFramTraining['tweet polarity']==4]
SplittedTrainSet = [dataFramTrainingNegative.iloc[0:8500,:],dataFramTrainingPositive.iloc[0:8500,:]]
#merging the two data sets
MergedArrays = pd.concat(SplittedTrainSet)
#defining polarity and labels
polarity = [ (MergedArrays['tweet polarity'] == 0), (MergedArrays['tweet polarity'] == 4)]
negative_positive = ['negative',  'positive']
#replacing polarity numberss with labels
MergedArrays['label'] = np.select(polarity, negative_positive)
#converting the pandas datafram to a dictionary
trainingset = MergedArrays.to_dict('records')

## PreProcessing Tweets

In [5]:
#class to clean the tweets
class TweetsPreProcessing:
    def __init__(self):
        # stop words are words that dont add much meaning to the sentence in the senitment
        # and punctiotation also don't add much meaning to the sentence in the senitment
        self.notimportantWords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL']) 
    #loops inside the tweet list and call the tweet cleaner function
    def TweetsProcessing(self, tweetlists):
        Tweets_Processed=[]
        for t in tweetlists:
            Tweets_Processed.append((self.TweetCleaner(t["text"]),t["label"]))  
        return Tweets_Processed
    # tweet cleaner function convert texts to lowercases and removes hashtags urls and usernames
    def TweetCleaner(self, t):
        urlLinks='((www\.[^\s]+)|(https?://[^\s]+))'
        username='@[^\s]+'
        hashtags= r'#([^\s]+)'
        t = t.lower()
        t = charRemoval.sub(urlLinks, 'URL', t) 
        t = charRemoval.sub(username, 'AT_USER', t)
        t = charRemoval.sub(hashtags, r'\1', t) 
        t = word_tokenize(t)
        wordcleaned=[]
        #loop through t and if the word is important then append it
        for w in t:
            if w not in self.notimportantWords:
                wordcleaned.append(w)
        return wordcleaned

## NLP using NLTK Libraries

In [6]:
def NLTKProcessor(tweet,Training_Data_Pre, command):
    #this part of the nltk proccessor checks if the command is to buildvocab
    if command== "buildvocab":
        #building vocabulary set
        vocab = []
        #for loop in the training data set and extend i
        for (i, j) in Training_Data_Pre:
            vocab.extend(i)
        #return those frequencies of vocab as keys.
        return nltk.FreqDist(vocab).keys()
     #this part of the nltk proccessor checks if the command is to extract features
    elif command=="feature":
        
        WordtweetFeatures=set(tweet)
        f={}
        for i in buildedvocab:
            #if word in the vocab is in the tweet and returns true false
            # using JSON key
            f['contains(%s)' % i]=(i in WordtweetFeatures)
        return f

## Loading the Training Module build by the other code

In [7]:
#load the training module that was build in the other code. 
classifier_f = open("./TrainModuleClassifierNLP.pickle", "rb")
classifierModel = pickle.load(classifier_f)
classifier_f.close()

## Generating Score and Reports

In [8]:
#Function to generate score, tweets table with labels, and export int excel file and text file
def generetaScoreAndReport():
    #count how many "negative" and "positive" words are in the array
    pcount=SKLearnResultLabels.count('positive')
    ncount=SKLearnResultLabels.count('negative')
    #generate the score 
    scoreP=str(100*pcount/len(SKLearnResultLabels)) +"%"
    scoreN=str(100*ncount/len(SKLearnResultLabels)) +"%"
    #if posiive is greate than negative than the tweet would be positive and vice versa
    if pcount > ncount :
        print('Overall Positive Sentiment for:{0} with the Positive Sentiment score of {1}'.format( key_word, scoreP))
    else: 
        print('Overall Positive Sentiment for:{0} with the Negative Sentiment score of {1}'.format( key_word, scoreN))    
    #defining variables
    Title = "Sentiment Analysis Report For Key Word: " + key_word + "\n"
    ReportTitle="SentimentAnalysisReport" + key_word+".xlsx"
    ReportTitle2="SentimentAnalysisReport" + key_word+".txt"
    TweetsFetched= "Tweets Retrieved: "+ str(len(testingset))+ "\n"
    PositveTweetsLabel= "Tweets Labeled as Positive: "+ str (pcount) +"\n"
    NegativeTweetsLabel= "Tweets Labeled as Negative: "+ str (ncount)+"\n"
    #print...
    print(Title)
    print(TweetsFetched)
    print(PositveTweetsLabel)
    print(NegativeTweetsLabel)
    print("Tweets Table: ")
    print()
    #write to a file
    file1 = open('./ReportOutput/' +ReportTitle2,"w")#write mode
    file1.write(Title)
    file1.write(TweetsFetched)
    file1.write(PositveTweetsLabel)
    file1.write(NegativeTweetsLabel)
    file1.close() 
    #Putting the tweets and labels in a table
    pd.set_option("display.max_rows", None, "display.max_columns", None,)
    pd.set_option('display.max_colwidth', None)
    tweet_text = []
    tweet_time= []
    ## for loop to append the texts
    for t in testingset:
        tweet_text.append(' '+t['text'])
        created= time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(t['time'],'%a %b %d %H:%M:%S +0000 %Y'))
        tweet_time.append(created)
    output_list = pd.DataFrame( {'Tweet': tweet_text,'Time': tweet_time,'Result': SKLearnResultLabels,'Search_Term': key_word})
    output_list.style.set_properties(subset=["col1", "col2"], **{'text-align': 'right'})
    output_list1 = output_list.style.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    output_list1.set_properties(**{'text-align': 'left'}).hide_index()
    #Extracting the table into a an excel file
    output_list1.to_excel('./ReportOutput/'+ ReportTitle)
    return output_list1


## The Program:

In [9]:
%%time
## USE CASE 1:Customer Enters a topic of interest
key_word= input("Please insert a keyword:  ")
##USE Case 2a :Software Connect to Twitter server to retrieve data
while key_word.isspace()==True or len(key_word)==0:
    key_word= input("Please insert a keyword: ")
testingset = []
testingset.extend(tweetsRetrieval(key_word))
while len(testingset)==0:
    key_word= input("Please insert another keyword that is proper: ")
    while key_word.isspace()==True or len(key_word)==0:
        key_word= input("Please insert a keyword: ")
    testingset = []
    testingset.extend(tweetsRetrieval(key_word))  
#USE Case 2b:Software Preprocess Data  
TestSet_PreProccesing = TweetsPreProcessing().TweetsProcessing(testingset)
TrainSet_PreProccesing = TweetsPreProcessing().TweetsProcessing(trainingset)
#USE Case 3: Software does nlp sentiment analysis on tweets
SKLearnResultLabels=[]
buildedvocab= NLTKProcessor(1,TrainSet_PreProccesing, "buildvocab")
for tweet in TestSet_PreProccesing:
    c= NLTKProcessor(tweet[0],1, "feature")
    label=classifierModel.classify(c)
    SKLearnResultLabels.append(label)
#USE Case 4 and t: Software produces Sentiment scores and generate a report
table=generetaScoreAndReport()
table
#Software send the customer/user report using the platform or email

Please insert a keyword:  Palestine
Retrieved tweets for the keyword (Palestine):  100 

Overall Positive Sentiment for:Palestine with the Positive Sentiment score of 74.0%
Sentiment Analysis Report For Key Word: Palestine

Tweets Retrieved: 100

Tweets Labeled as Positive: 74

Tweets Labeled as Negative: 26

Tweets Table: 

CPU times: user 3.36 s, sys: 12.8 ms, total: 3.37 s
Wall time: 9.71 s


Tweet,Time,Result,Search_Term
"RT @desert____lion: Just 7k will be enough to make clean water available to dozens of vulnerable kids in Palestine 🇵🇸, Yemen 🇾🇪, Syria 🇸🇾,…",2020-12-07 20:55:28,negative,Palestine
@benedictrogers It's funny you say this but you say the exact opposite about the weaponising of #Antisemitism and the closing down of criticism of Israel about Palestine. You are a hypocrite and the world sees it. #ShiaMasot https://t.co/fTSWjhrntD,2020-12-07 20:55:09,positive,Palestine
"RT @Act4PalGlobal: @swilkinsonbc Been another shooting: ""Palestinian teen Omar Ahmad Mahmoud was shot in his eye this evening by Zionist oc…",2020-12-07 20:55:01,negative,Palestine
"@marwilliamson But what about all the money the establishment @SpeakerPelosi @SenSchumer @senatemajldr @realDonaldTrump @JoeBiden @KamalaHarris, & the warmongers in Washington DC need for the MIC, illegal #RegimeChanges, interference in sovereign nations and Israeli occupation of Palestine???",2020-12-07 20:55:01,positive,Palestine
In the list of things which concern me; the plight of Israel and Palestine is petty bloody low down. I honestly couldn’t give a toss what they do to each other. We have our own problems closer to home 🤷🏻‍♂️,2020-12-07 20:54:55,negative,Palestine
RT @dcmpsey: putting this on the tl again please actually read it and educate yourself https://t.co/Aufh4PKJgK,2020-12-07 20:54:43,positive,Palestine
"RT @178kakapo: ░▒▓► Event for 🌎🌍🌏&🇵🇸 ➡ Dec. 11, 2020 ❝ Online Event: The Importance of the Student Movement in Confronting Colonialism ▼F…",2020-12-07 20:54:42,positive,Palestine
"But what about all the money the establishment @SpeakerPelosi @SenSchumer @senatemajldr @realDonaldTrump @JoeBiden @KamalaHarris, & the warmongers in Washington DC need for the MIC, illegal #RegimeChanges, interference in sovereign nations and Israeli occupation of Palestine??? https://t.co/LM9JiMemDe",2020-12-07 20:54:27,positive,Palestine
@freddiechurro13 @OneADONAIOne @Deadlycause How can you deny Palestine then? VERY Strange https://t.co/8RRyxj997V,2020-12-07 20:54:25,positive,Palestine
RT @PalBint: Israel is determined to annex all of the WB and EJ knowing full well the international community will continue to look the oth…,2020-12-07 20:53:58,positive,Palestine
